# Tuning BP parameters programmatically

The process will evaluate find the best combination of `ms_scaling_factor` and `max_iter`.
- ms_scaling_factor: selecting from $\{ 0.5, 0.625, 0.8, 1.0 \}$ to cover common parameters in existing literature.
- max_iter: selecting from $\{ 1~10, 12, 16, 20, 50, 100, 200, 500, 1000 \}$, a total of 18 choices
The total number of choices is then 4 * 18 = 72, still quite expensive.

We need to reduce the number of samples to reach a fair comparison.
To do this, we ensure that all decoders will use exactly the same set of syndrome to evaluate.

To execute this notebook with a custom code, noise and decoder, use
```sh
papermill bp-tuner.ipynb 
```

In [ ]:
decoder: str = "bposd(osd_order@0)"
code: str = "rsc(d@3,p@0.01)"
noise: str = "none"

ms_scaling_factor_choices: list[float] = [0.5, 0.626, 0.8, 1.0]
max_iter_choices: list[int] = list(range(1, 11)) + [12, 16, 20, 50, 100, 200, 500, 1000]

max_cpu_hours: float = 1.0
target_precision: float = 0.04  # about 4000 errors for the configuration with the smallest 

slurm_maximum_jobs: int = 50  # start with a smaller number of workers to avoid resource waste
slurm_cores_per_node: int = 10  # (slurm_maximum_jobs // slurm_cores_per_node) should not exceed 200
slurm_mem_per_job: int = 4  # 4GB per job
slurm_extra: dict = dict(
    walltime = "1-00:00:00",  # adaptively shutdown if no more jobs
    queue = "scavenge",  # use with caution: dask does not seem to handle scavenge workers well
    job_extra_directives = ["--requeue"],  # use with scavenge partition will help spawn scavenged jobs
)

json_filename: str | None = None
force_finished: bool = False  # only plot the figure and do not run experiments

In [ ]:
decoder = decoder.replace("@", "=")
code = code.replace("@", "=")
noise = noise.replace("@", "=")

from slugify import slugify
from dotmap import DotMap as dmap

if json_filename is None:
    json_filename = (
        "z-bp-tuner-"
        + slugify(code)
        + "."
        + slugify(noise)
        + "."
        + slugify(decoder)
        + ".json"
    )
print(json_filename)